In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix
from xgboost import XGBClassifier
import numpy as np
import statistics

In [2]:
np.random.seed(0)

In [3]:
df_source_annot_test = pd.read_csv("gene_annot_test.tsv", sep="\t")

In [4]:
df_annot_test = df_source_annot_test.copy()

### We can notice that, apart from intron_sources, most of the features can be transformed to binary using Sklearn's LabelEncoder.
We can see from the unique labels of intron_sources that there are only 3 types

In [5]:
df_annot_test.intron_sources.unique()

array(['SLR,CLS', 'CLS', 'SLR', 'SLR,RAC', 'SLR,CLS,RAC', 'RAC',
       'CLS,RAC'], dtype=object)

In [6]:
df_annot_test["intron_sources_SLR"] = df_annot_test["intron_sources"].str.contains("SLR").astype(int)
df_annot_test["intron_sources_CLS"] = df_annot_test["intron_sources"].str.contains("CLS").astype(int)
df_annot_test["intron_sources_RAC"] = df_annot_test["intron_sources"].str.contains("RAC").astype(int)
df_annot_test

,coords,outcome,score,length,prev_annot,transcript_source,intron_sources,splice_site,repeat_overlap,ss_antisense,...,opp_strand,false_ret_int,transcript_id,gtype,bbiotype,rel_int_sup,rel_int_sup_k,intron_sources_SLR,intron_sources_CLS,intron_sources_RAC
0,chr1:261635-267302:-1,accepted,888,5668,yes,SLRseq,"SLR,CLS",GT..AG,Type I Transposons/LINE,no,...,no,no,OTTHUMT00000499557,transcribed_processed_pseudogene,non-coding,0.311999,0.311999,1,1,0
1,chr1:259026-261549:-1,accepted,650,2524,yes,SLRseq,"SLR,CLS",GT..AG,Type I Transposons/LINE,no,...,no,no,OTTHUMT00000499557,transcribed_processed_pseudogene,non-coding,-0.311999,-0.311999,1,1,0
2,chr1:732208-739802:-1,rejected,0,7595,no,PacBio Capture-seq,CLS,GT..AG,Type I Transposons/SINE,no,...,no,no,OTTHUMT00000500170,transcribed_processed_pseudogene,non-coding,-2.525729,-2.708050,0,1,0
3,chr1:720201-732016:-1,accepted,0,11816,yes,PacBio Capture-seq,CLS,GT..AG,Type II Transposons,no,...,no,no,OTTHUMT00000500170,transcribed_processed_pseudogene,non-coding,-2.525729,-2.931194,0,1,0
4,chr1:711923-720031:-1,accepted,27,8109,yes,PacBio Capture-seq,CLS,GT..AG,No overlap,no,...,no,no,OTTHUMT00000500170,transcribed_processed_pseudogene,non-coding,1.216395,0.810930,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11005,chrY:20582694-20584473:1,accepted,278936,1780,yes,PacBio Capture-seq,"SLR,CLS,RAC",GT..AG,No overlap,no,...,no,no,OTTHUMT00000500440,protein_coding,coding,-0.141738,-0.364851,1,1,1
11006,chrY:20584525-20588023:1,accepted,286043,3499,yes,PacBio Capture-seq,"SLR,CLS,RAC",GT..AG,No overlap,no,...,no,no,OTTHUMT00000500440,protein_coding,coding,-0.112146,-0.335259,1,1,1
11007,chrY:20588106-20589483:1,accepted,444721,1378,yes,PacBio Capture-seq,"SLR,CLS,RAC",GT..AG,No overlap,no,...,no,no,OTTHUMT00000500440,protein_coding,coding,0.433606,0.210496,1,1,1
11008,chrY:20589576-20592340:1,accepted,468983,2765,yes,PacBio Capture-seq,"SLR,CLS,RAC",GT..AG,No overlap,no,...,no,no,OTTHUMT00000500440,protein_coding,coding,0.503702,0.280593,1,1,1


### We convert "accepted" and "rejected" to 1s and 0s, this facilitates the calculation of ROC curve

In [7]:

for col in df_annot_test.drop(["coords"], axis=1).columns:
    le = preprocessing.LabelEncoder()
    le.fit(df_annot_test[col])
    df_annot_test[col] = le.transform(df_annot_test[col])

df_annot_test

,coords,outcome,score,length,prev_annot,transcript_source,intron_sources,splice_site,repeat_overlap,ss_antisense,...,opp_strand,false_ret_int,transcript_id,gtype,bbiotype,rel_int_sup,rel_int_sup_k,intron_sources_SLR,intron_sources_CLS,intron_sources_RAC
0,chr1:261635-267302:-1,0,696,3524,1,3,4,2,9,0,...,0,0,823,6,1,7185,7199,1,1,0
1,chr1:259026-261549:-1,0,558,2130,1,3,4,2,9,0,...,0,0,823,6,1,3479,3283,1,1,0
2,chr1:732208-739802:-1,1,0,4005,0,1,0,2,20,0,...,0,0,1434,6,1,1629,1496,0,1,0
3,chr1:720201-732016:-1,0,0,4609,1,1,0,2,22,0,...,0,0,1434,6,1,1629,1399,0,1,0
4,chr1:711923-720031:-1,0,26,4092,1,1,0,2,2,0,...,0,0,1434,6,1,9330,8260,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11005,chrY:20582694-20584473:1,0,5548,1593,1,1,5,2,2,0,...,0,0,1703,4,0,4092,3128,1,1,1
11006,chrY:20584525-20588023:1,0,5595,2703,1,1,5,2,2,0,...,0,0,1703,4,0,4275,3207,1,1,1
11007,chrY:20588106-20589483:1,0,6348,1256,1,1,5,2,2,0,...,0,0,1703,4,0,7867,6679,1,1,1
11008,chrY:20589576-20592340:1,0,6428,2279,1,1,5,2,2,0,...,0,0,1703,4,0,8162,7043,1,1,1


In [8]:
df_annot_test.columns

Index(['coords', 'outcome', 'score', 'length', 'prev_annot',
       'transcript_source', 'intron_sources', 'splice_site', 'repeat_overlap',
       'ss_antisense', 'rej_reason', 'annot_match', 'incorrect_locus',
       'opp_strand', 'false_ret_int', 'transcript_id', 'gtype', 'bbiotype',
       'rel_int_sup', 'rel_int_sup_k', 'intron_sources_SLR',
       'intron_sources_CLS', 'intron_sources_RAC'],
      dtype='object')

### We then select all the relevant column(s) for the input features and the label

In [9]:
df_annot_test_y = df_annot_test["outcome"]

# Drop the columns that might not be available prior to manual gene annotation or irrelevant to ML
df_annot_test_X_1 = df_annot_test.drop(["coords", "outcome", "transcript_id", "gtype", "rej_reason"], axis=1)
df_annot_test_X_1

,score,length,prev_annot,transcript_source,intron_sources,splice_site,repeat_overlap,ss_antisense,annot_match,incorrect_locus,opp_strand,false_ret_int,bbiotype,rel_int_sup,rel_int_sup_k,intron_sources_SLR,intron_sources_CLS,intron_sources_RAC
0,696,3524,1,3,4,2,9,0,1,0,0,0,1,7185,7199,1,1,0
1,558,2130,1,3,4,2,9,0,1,0,0,0,1,3479,3283,1,1,0
2,0,4005,0,1,0,2,20,0,0,0,0,0,1,1629,1496,0,1,0
3,0,4609,1,1,0,2,22,0,0,0,0,0,1,1629,1399,0,1,0
4,26,4092,1,1,0,2,2,0,0,0,0,0,1,9330,8260,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11005,5548,1593,1,1,5,2,2,0,1,0,0,0,0,4092,3128,1,1,1
11006,5595,2703,1,1,5,2,2,0,1,0,0,0,0,4275,3207,1,1,1
11007,6348,1256,1,1,5,2,2,0,1,0,0,0,0,7867,6679,1,1,1
11008,6428,2279,1,1,5,2,2,0,1,0,0,0,0,8162,7043,1,1,1


# 1. We first test the cross-validation performance of this dataset without any feature engineering

In [10]:
# Split the dataset to 9:1 Train/Val and Test set
X_train_val, X_test, y_train_val, y_test = train_test_split(df_annot_test_X_1, df_annot_test_y, stratify=df_annot_test_y, test_size=0.1, shuffle=True)

In [11]:
cv_acc_list = []
cv_ba_acc_list = []
cv_rocauc_list = []
cv_precision_list = []
cv_mcc_list = []
cv_specificity_list = []
cv_sensitivity_list = []

In [12]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X_train_val, y_train_val)

for train_index, val_index in skf.split(X_train_val, y_train_val):
    X_train, X_val = X_train_val.iloc[train_index], X_train_val.iloc[val_index]
    y_train, y_val = y_train_val.iloc[train_index].to_numpy().flatten(), y_train_val.iloc[val_index].to_numpy().flatten()

    xgboost_clf = XGBClassifier(seed=0)
    xgboost_clf.fit(X_train, y_train)

    y_predict = xgboost_clf.predict_proba(X_val)
    y_predict = y_predict[:, 1]
    y_predict_class = list(map(round, y_predict))

    test_acc = accuracy_score(y_val, y_predict_class)
    test_rocauc = roc_auc_score(y_val, y_predict)
    test_bal_acc = balanced_accuracy_score(y_val, y_predict_class)
    test_precision = precision_score(y_val, y_predict_class)  # tp/(tp+fp)
    test_mcc = matthews_corrcoef(y_val, y_predict_class)
    tn, fp, fn, tp = confusion_matrix(y_val, y_predict_class).ravel()

    # Adding the metrics to their list
    cv_acc_list.append(test_acc)
    cv_ba_acc_list.append(test_bal_acc)
    cv_rocauc_list.append(test_rocauc)
    cv_precision_list.append(test_precision)
    cv_mcc_list.append(test_mcc)
    cv_specificity_list.append(tn / (tn + fp))
    cv_sensitivity_list.append(tp / (fn + tp))

# return cv_acc_list, cv_ba_acc_list, cv_rocauc_list, cv_precision_list, cv_mcc_list, cv_specificity_list, cv_sensitivity_list

In [13]:
print("------------------------------------------------------------------------------------------")
print("Stratified Cross-Validation Performance")
print("------------------------------------------------------------------------------------------")
print("Accuracy: %s \nAUCROC: %s \nMCC: %s \nSensitivity: %s \nSpecificity: %s \nBalanced Accuracy: %s" % (
        statistics.mean(cv_acc_list), statistics.mean(cv_rocauc_list), statistics.mean(cv_mcc_list),
        statistics.mean(cv_sensitivity_list), statistics.mean(cv_specificity_list), statistics.mean(cv_ba_acc_list)))

print("------------------------------------------------------------------------------------------")
print("Accuracy SD: %s \nAUCROC SD: %s \nMCC SD: %s \nSensitivity SD: %s \nSpecificity SD: %s \nBalanced Accuracy SD: %s" % (
        np.std(cv_acc_list), np.std(cv_rocauc_list), np.std(cv_mcc_list),
        np.std(cv_sensitivity_list), np.std(cv_specificity_list), np.std(cv_ba_acc_list)))

------------------------------------------------------------------------------------------
Stratified Cross-Validation Performance
------------------------------------------------------------------------------------------
Accuracy: 0.9776966958048994 
AUCROC: 0.9847982221251834 
MCC: 0.6797873515281583 
Sensitivity: 0.6655312847093668 
Specificity: 0.9897282320421967 
Balanced Accuracy: 0.8276297583757818
------------------------------------------------------------------------------------------
Accuracy SD: 0.003915644493521446 
AUCROC SD: 0.0020777466320006403 
MCC SD: 0.05079253541609124 
Sensitivity SD: 0.03997531656436102 
Specificity SD: 0.003341844843864175 
Balanced Accuracy SD: 0.02055704193838945


# 2. We then test the cross-validation performance of this dataset *WITH* some feature engineering

In [15]:
#TODO